**Задание 1**

In [1]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [2]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
f = open('corpus_hw2.txt', 'r', encoding = 'utf-8')
text = f.read()

In [5]:
!C:/Users/Asus/Sem1_avtobreya/udpipe-1.2.0-bin/bin-win64/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:/Users/Asus/Sem1_avtobreya/russian-syntagrus-ud-2.4-190531.udpipe \
< corpus_hw2.txt > corpus_hw2.conllu

Loading UDPipe model: done.


In [4]:
import nltk
from nltk.parse import DependencyGraph
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [7]:
from nltk.corpus import stopwords
stop = stopwords.words('russian')

In [8]:
trees = []
with open('corpus_hw2.conllu', encoding = 'utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [9]:
trees_update = []
for i in trees:
    if i != '':
        trees_update.append(i)

In [10]:
_FILTER_RELS = ['punct', 'conj', 'parataxis']
def get_subtree(nodes, node):
    if not nodes[node]['deps']:
        return [node]
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] 
                         if rel not in _FILTER_RELS
                         for dep in nodes[node]['deps'][rel]]

In [11]:
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

In [147]:
result = []
for t in trees_update:
    try:
        g = DependencyGraph(t, top_relation_label='root')
    except AssertionError:
        continue
    np_list = []
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
            np = list(sorted(flatten(get_subtree(g.nodes, n))))
            np_list.append(
                ' '.join([g.nodes[i]['word'] for i in np])
            )
    result.append(np_list)

In [148]:
result

[['Я предложил бы Единой России объединить в одном окне не только депутатов',
  'Единой России объединить в одном окне не только депутатов',
  'заявил Ъ член политсовета СПС Борис Надеждин'],
 ['И тогда о разделении властей придется окончательно забыть',
  'окончательно забыть'],
 ['20 ноября он подал в арбитражный суд Москвы иск в котором просит признать недействительным договор о присоединении Импэксбанка к Райффайзенбанку',
  'в котором просит признать недействительным договор о присоединении Импэксбанка к Райффайзенбанку',
  'признать недействительным договор о присоединении Импэксбанка к Райффайзенбанку'],
 ['Член ЦИКа от СПС Вадим Прохоров заявил что ЦИК не стал запускать механизм снятия партии с выборов хотя накануне он предполагал что представители администрации президента как следует поработают с членами ЦИКа',
  'что ЦИК не стал запускать механизм снятия партии с выборов хотя накануне он предполагал что представители администрации президента как следует поработают с членами Ц

In [150]:
text_for_analize = []
for i in result:
    if i != []:
        for j in i:
            text_for_analize.append(j)
            new_text = ' '.join(text_for_analize)

In [151]:
new_text

"Я предложил бы Единой России объединить в одном окне не только депутатов Единой России объединить в одном окне не только депутатов заявил Ъ член политсовета СПС Борис Надеждин И тогда о разделении властей придется окончательно забыть окончательно забыть 20 ноября он подал в арбитражный суд Москвы иск в котором просит признать недействительным договор о присоединении Импэксбанка к Райффайзенбанку в котором просит признать недействительным договор о присоединении Импэксбанка к Райффайзенбанку признать недействительным договор о присоединении Импэксбанка к Райффайзенбанку Член ЦИКа от СПС Вадим Прохоров заявил что ЦИК не стал запускать механизм снятия партии с выборов хотя накануне он предполагал что представители администрации президента как следует поработают с членами ЦИКа что ЦИК не стал запускать механизм снятия партии с выборов хотя накануне он предполагал что представители администрации президента как следует поработают с членами ЦИКа запускать механизм снятия партии с выборов хот

In [152]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [m.parse(t)[0].normal_form for t in tokens]
    return lemmas

In [174]:
def get_ngrams_stopwords(tokens, n=2, patterns=None, stoplist=[]):
    tokens = list(filter(lambda x: x not in stoplist, tokens))
    ngrams = []
    for i in range(len(tokens) - (n - 1)):
        ngram = tokens[i:i+n]
        tags = [m.parse(t)[0].tag.POS for t in ngram]
        if patterns is not None:
            if tags in patterns:
                ngrams.append(tuple(ngram))
        else:
            ngrams.append(tuple(ngram))
    ### YOUR CODE HERE
    return ngrams

In [175]:
from collections import Counter

In [176]:
def collect_stats(text):
    word_counter = Counter()
    word_counter.update(text)
    
    return word_counter

In [177]:
count_words = new_text.split(' ')

In [178]:
stats = collect_stats(count_words)

In [183]:
bi_grams = get_ngrams_stopwords(normalize(new_text), patterns=[['INFN', 'NOUN']], stoplist = stop)

In [184]:
bi_grams

[('объединить', 'окно'),
 ('объединить', 'окно'),
 ('заявить', 'цик'),
 ('запускать', 'механизм'),
 ('предполагать', 'представитель'),
 ('поработать', 'член'),
 ('запускать', 'механизм'),
 ('предполагать', 'представитель'),
 ('поработать', 'член'),
 ('запускать', 'механизм'),
 ('предполагать', 'представитель'),
 ('поработать', 'член'),
 ('предполагать', 'представитель'),
 ('поработать', 'член'),
 ('поработать', 'член'),
 ('поработать', 'член'),
 ('последовать', 'реакция'),
 ('обратиться', 'председатель'),
 ('рассматривать', 'дело'),
 ('рассматривать', 'дело'),
 ('рассматривать', 'дело'),
 ('рассматривать', 'дело'),
 ('рассказать', 'понедельник'),
 ('рассмотреть', 'вопрос'),
 ('рассмотреть', 'вопрос'),
 ('отмечать', 'репутация'),
 ('освободить', 'стража'),
 ('взять', 'стража'),
 ('отстранить', 'должность'),
 ('увидеть', 'стенд'),
 ('определяться', 'координата'),
 ('терпеть', 'бедствие'),
 ('определяться', 'координата'),
 ('терпеть', 'бедствие'),
 ('терпеть', 'бедствие'),
 ('начаться', '

In [186]:
high_frequency_verbs = []
for key in stats:
    part_of_speech = m.parse(key)[0].tag.POS
    if part_of_speech == 'INFN' and stats[key] >= 50:
        high_frequency_verbs.append(normalize(key))

In [187]:
high_frequency_verbs

[['признать'],
 ['освободить'],
 ['обжаловать'],
 ['выплатить'],
 ['вернуть'],
 ['быть'],
 ['доказать'],
 ['взыскать'],
 ['оспорить'],
 ['подать'],
 ['отменить'],
 ['обратиться']]

In [196]:
colloc = []
for i in bi_grams:
    for j in high_frequency_verbs:
        if i[0] == j[0] and i not in colloc:
            colloc.append(i)

In [198]:
colloc #список глагольных коллокаций

[('обратиться', 'председатель'),
 ('освободить', 'стража'),
 ('освободить', 'залог'),
 ('обжаловать', 'постановление'),
 ('обратиться', 'иск'),
 ('признать', 'сделка'),
 ('освободить', 'е'),
 ('подать', 'суд'),
 ('выплатить', 'компенсация'),
 ('вернуть', 'инвалид'),
 ('обжаловать', 'санкция'),
 ('подать', 'заявка'),
 ('подать', 'ветеран'),
 ('освободить', 'решение'),
 ('взыскать', 'касьянов'),
 ('подать', 'иск'),
 ('обжаловать', 'решение'),
 ('обжаловать', 'суд'),
 ('доказать', 'любой'),
 ('доказать', 'сведение'),
 ('подать', 'голос'),
 ('признать', 'собрание'),
 ('обратиться', 'защитник'),
 ('вернуть', 'почта'),
 ('обратиться', 'суд'),
 ('признать', 'адамов'),
 ('оспорить', 'суд'),
 ('оспорить', 'сделка'),
 ('признать', 'бабаков'),
 ('обжаловать', 'отказ'),
 ('признать', 'право'),
 ('оспорить', 'решение'),
 ('доказать', 'суд'),
 ('признать', 'действие'),
 ('подать', 'ходатайство'),
 ('вернуть', 'долг'),
 ('подать', 'жалоба'),
 ('подать', 'роснефть'),
 ('вернуть', 'суд'),
 ('доказать',

**Задание 2**

In [17]:
import nltk
from nltk.collocations import BigramCollocationFinder

In [18]:
texts = normalize(new_text)

**log-likelihood**

In [214]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents(colloc)

In [217]:
likelihood = finder2.nbest(bigram_measures.likelihood_ratio, 100)

In [218]:
likelihood #вот это неплохо

[('освободить', 'должность'),
 ('освободить', 'дом'),
 ('освободить', 'е'),
 ('освободить', 'евгений'),
 ('освободить', 'зал'),
 ('освободить', 'залог'),
 ('освободить', 'подписка'),
 ('освободить', 'стража'),
 ('оспорить', 'акт'),
 ('оспорить', 'выбор'),
 ('оспорить', 'доначисление'),
 ('оспорить', 'итог'),
 ('оспорить', 'мтс'),
 ('оспорить', 'партия'),
 ('оспорить', 'правомерность'),
 ('взыскать', 'авиаперевозчик'),
 ('взыскать', 'аренда'),
 ('взыскать', 'банка'),
 ('взыскать', 'бюджет'),
 ('взыскать', 'ведомство'),
 ('взыскать', 'доход'),
 ('взыскать', 'касьянов'),
 ('взыскать', 'концерн'),
 ('взыскать', 'мнпз'),
 ('взыскать', 'предприятие'),
 ('отменить', 'визит'),
 ('отменить', 'вторник'),
 ('отменить', 'гост'),
 ('отменить', 'день'),
 ('отменить', 'запрет'),
 ('отменить', 'мера'),
 ('отменить', 'предупреждение'),
 ('отменить', 'разрешение'),
 ('отменить', 'регистрация'),
 ('отменить', 'цена'),
 ('вернуть', 'актив'),
 ('вернуть', 'дача'),
 ('вернуть', 'дело'),
 ('вернуть', 'долг')

**PMI**

In [219]:
pmi = finder2.score_ngrams(bigram_measures.pmi)

In [220]:
pmi

[(('освободить', 'должность'), 5.708951217996608),
 (('освободить', 'дом'), 5.708951217996608),
 (('освободить', 'е'), 5.708951217996608),
 (('освободить', 'евгений'), 5.708951217996608),
 (('освободить', 'зал'), 5.708951217996608),
 (('освободить', 'залог'), 5.708951217996608),
 (('освободить', 'подписка'), 5.708951217996608),
 (('освободить', 'стража'), 5.708951217996608),
 (('оспорить', 'акт'), 5.321928094887361),
 (('оспорить', 'выбор'), 5.321928094887361),
 (('оспорить', 'доначисление'), 5.321928094887361),
 (('оспорить', 'итог'), 5.321928094887361),
 (('оспорить', 'мтс'), 5.321928094887361),
 (('оспорить', 'партия'), 5.321928094887361),
 (('оспорить', 'правомерность'), 5.321928094887361),
 (('взыскать', 'авиаперевозчик'), 5.087462841250338),
 (('взыскать', 'аренда'), 5.087462841250338),
 (('взыскать', 'банка'), 5.087462841250338),
 (('взыскать', 'бюджет'), 5.087462841250338),
 (('взыскать', 'ведомство'), 5.087462841250338),
 (('взыскать', 'доход'), 5.087462841250338),
 (('взыскат

In [221]:
pmi_sorted = sorted([x for x in pmi if x[1] != 1.0], key=lambda x: x[1], reverse=True)[:100] #тоже неплохо

In [222]:
pmi_sorted

[(('освободить', 'должность'), 5.708951217996608),
 (('освободить', 'дом'), 5.708951217996608),
 (('освободить', 'е'), 5.708951217996608),
 (('освободить', 'евгений'), 5.708951217996608),
 (('освободить', 'зал'), 5.708951217996608),
 (('освободить', 'залог'), 5.708951217996608),
 (('освободить', 'подписка'), 5.708951217996608),
 (('освободить', 'стража'), 5.708951217996608),
 (('оспорить', 'акт'), 5.321928094887361),
 (('оспорить', 'выбор'), 5.321928094887361),
 (('оспорить', 'доначисление'), 5.321928094887361),
 (('оспорить', 'итог'), 5.321928094887361),
 (('оспорить', 'мтс'), 5.321928094887361),
 (('оспорить', 'партия'), 5.321928094887361),
 (('оспорить', 'правомерность'), 5.321928094887361),
 (('взыскать', 'авиаперевозчик'), 5.087462841250338),
 (('взыскать', 'аренда'), 5.087462841250338),
 (('взыскать', 'банка'), 5.087462841250338),
 (('взыскать', 'бюджет'), 5.087462841250338),
 (('взыскать', 'ведомство'), 5.087462841250338),
 (('взыскать', 'доход'), 5.087462841250338),
 (('взыскат

**dice**

In [223]:
scores = finder2.score_ngrams(bigram_measures.dice)

In [224]:
scores

[(('освободить', 'должность'), 0.14285714285714285),
 (('освободить', 'дом'), 0.14285714285714285),
 (('освободить', 'е'), 0.14285714285714285),
 (('освободить', 'евгений'), 0.14285714285714285),
 (('освободить', 'зал'), 0.14285714285714285),
 (('освободить', 'залог'), 0.14285714285714285),
 (('освободить', 'подписка'), 0.14285714285714285),
 (('освободить', 'стража'), 0.14285714285714285),
 (('освободить', 'окруашвили'), 0.13333333333333333),
 (('освободить', 'сергей'), 0.13333333333333333),
 (('освободить', 'арест'), 0.11764705882352941),
 (('освободить', 'господин'), 0.11764705882352941),
 (('оспорить', 'акт'), 0.1111111111111111),
 (('оспорить', 'выбор'), 0.1111111111111111),
 (('оспорить', 'доначисление'), 0.1111111111111111),
 (('оспорить', 'итог'), 0.1111111111111111),
 (('оспорить', 'мтс'), 0.1111111111111111),
 (('оспорить', 'партия'), 0.1111111111111111),
 (('оспорить', 'правомерность'), 0.1111111111111111),
 (('освободить', 'решение'), 0.10526315789473684),
 (('оспорить', 'а

In [225]:
dice_sorted = sorted([x for x in scores if x[1] != 1.0], key=lambda x: x[1], reverse=True)[:100] #тоже неплохо

In [226]:
dice_sorted

[(('освободить', 'должность'), 0.14285714285714285),
 (('освободить', 'дом'), 0.14285714285714285),
 (('освободить', 'е'), 0.14285714285714285),
 (('освободить', 'евгений'), 0.14285714285714285),
 (('освободить', 'зал'), 0.14285714285714285),
 (('освободить', 'залог'), 0.14285714285714285),
 (('освободить', 'подписка'), 0.14285714285714285),
 (('освободить', 'стража'), 0.14285714285714285),
 (('освободить', 'окруашвили'), 0.13333333333333333),
 (('освободить', 'сергей'), 0.13333333333333333),
 (('освободить', 'арест'), 0.11764705882352941),
 (('освободить', 'господин'), 0.11764705882352941),
 (('оспорить', 'акт'), 0.1111111111111111),
 (('оспорить', 'выбор'), 0.1111111111111111),
 (('оспорить', 'доначисление'), 0.1111111111111111),
 (('оспорить', 'итог'), 0.1111111111111111),
 (('оспорить', 'мтс'), 0.1111111111111111),
 (('оспорить', 'партия'), 0.1111111111111111),
 (('оспорить', 'правомерность'), 0.1111111111111111),
 (('освободить', 'решение'), 0.10526315789473684),
 (('оспорить', 'а

**Задание 3**

In [237]:
gold1 = []
for i in likelihood:
    for j in pmi_sorted:
        if i == j[0] and i not in gold1:
            gold1.append(i)

In [239]:
gold = []
for i in gold1:
    for j in dice_sorted:
        if i == j[0] and i not in gold:
            gold.append(i) #неповторяющиеся коллокации из нашего текста

In [265]:
gold

[('освободить', 'должность'),
 ('освободить', 'дом'),
 ('освободить', 'е'),
 ('освободить', 'евгений'),
 ('освободить', 'зал'),
 ('освободить', 'залог'),
 ('освободить', 'подписка'),
 ('освободить', 'стража'),
 ('оспорить', 'акт'),
 ('оспорить', 'выбор'),
 ('оспорить', 'доначисление'),
 ('оспорить', 'итог'),
 ('оспорить', 'мтс'),
 ('оспорить', 'партия'),
 ('оспорить', 'правомерность'),
 ('взыскать', 'авиаперевозчик'),
 ('взыскать', 'аренда'),
 ('взыскать', 'банка'),
 ('взыскать', 'бюджет'),
 ('взыскать', 'ведомство'),
 ('взыскать', 'доход'),
 ('взыскать', 'касьянов'),
 ('взыскать', 'концерн'),
 ('взыскать', 'мнпз'),
 ('взыскать', 'предприятие'),
 ('отменить', 'визит'),
 ('отменить', 'вторник'),
 ('отменить', 'гост'),
 ('отменить', 'день'),
 ('отменить', 'запрет'),
 ('отменить', 'мера'),
 ('отменить', 'предупреждение'),
 ('отменить', 'разрешение'),
 ('отменить', 'регистрация'),
 ('отменить', 'цена'),
 ('вернуть', 'актив'),
 ('вернуть', 'дача'),
 ('вернуть', 'дело'),
 ('вернуть', 'долг')

In [248]:
f1 = open('verb_coll.txt', 'r', encoding = 'utf-8')
text_gold = f1.read()

In [251]:
text_gold[:1000]

'аборт\tbase=действие mood=объект order=NV\tаборт делать\nаборт\tbase=действие mood=объект order=VN\tделать аборт\nаборт\tbase=действие mood=объект order=VN\tсделать аборт\nавария\tbase=существование/создание mood=субъект semclass=факт order=NV\tавария произошла\nавария\tbase=существование/создание mood=субъект semclass=факт order=NV\tавария случилась\nавария\tbase=действие mood=объект order=VN\tпотерпеть аварию\nавария\tbase=существование/создание mood=субъект semclass=факт order=VN\tпроизошла авария\nавария\tbase=существование/создание mood=субъект semclass=факт order=VN\tслучилась авария\nавторитет\tbase=обладание phase=начало order=VN\tзавоевать авторитет\nавторитет\tsemclass=содействие subclass=вред order=VN\tподорвать авторитет\nавторитет\tsemclass=содействие subclass=вред order=VN\tподорвать авторитет\nавторитет\tbase=обладание mood=непрямой объект order=VN\tпользоваться авторитетом\nавторитет\tbase=обладание mood=непрямой объект order=VN\tпользоваться большим авторитетом\nавтор

In [259]:
import re
gold_colloc = re.findall(r'order=VN\t\w+ \w+\n', text_gold)

In [263]:
gold_coll = []
for i in gold_colloc:
    gold_coll.append(re.findall(r'\w+ \w+', str(i))) #коллокации из файла

In [269]:
gold_coll

[['делать аборт'],
 ['сделать аборт'],
 ['потерпеть аварию'],
 ['произошла авария'],
 ['случилась авария'],
 ['завоевать авторитет'],
 ['подорвать авторитет'],
 ['подорвать авторитет'],
 ['пользоваться авторитетом'],
 ['оспаривать акт'],
 ['проявить активность'],
 ['проявить активность'],
 ['проявлять активность'],
 ['проявлять активность'],
 ['делать акцент'],
 ['расставлять акценты'],
 ['расставлять акценты'],
 ['расставлять акценты'],
 ['сделать акцент'],
 ['сместить акценты'],
 ['сместить акценты'],
 ['предлагать алгоритм'],
 ['предлагать алгоритм'],
 ['разработать алгоритм'],
 ['разработать алгоритм'],
 ['вызывать аллергию'],
 ['вызывать аллергию'],
 ['найти альтернативу'],
 ['найти альтернативу'],
 ['поддаваться анализу'],
 ['поддаваться анализу'],
 ['подлежать анализу'],
 ['показал анализ'],
 ['показывает анализ'],
 ['посвятить анализу'],
 ['посвятить анализу'],
 ['предполагать анализ'],
 ['провести анализ'],
 ['провести анализ'],
 ['сделать анализ'],
 ['провести аналогию'],
 ['

Осталось пересечь полученные коллокации, но не успеваю до дедлайна :(